In [6]:
import pandas as pd
import numpy as np
import re
import json
import pickle

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F

In [3]:
from tqdm import tqdm
from tqdm import tqdm_notebook
from tqdm import notebook

In [5]:
ls

data_category.pkl  sample_data/


In [189]:
data=pickle.load(open('data_category.pkl','rb'))

In [119]:
data

Name
! Bien Buro Installatie en Netwerken B.V.            shopping
! Manna                                      service_provider
! zaak ! MAGEry                                      shopping
!Bewust sociaal op web                       service_provider
!DRAMA!                                      service_provider
                                                   ...       
önder Biskin                                   transportation
özmes                                                industry
ø audiovisueel                                       industry
ŠKODA Autohaas                                 transportation
€5,- Shop                                            shopping
Name: Category0, Length: 1731940, dtype: object

In [7]:
data.shape

(342781, 6)

In [10]:
data.isna().sum()

0

In [11]:
set(data)

{'animal_care',
 'association/political_org/union',
 'bank',
 'eating_out/takeaway',
 'education/training',
 'elderly care',
 'entertainment',
 'finance/credit',
 'food_and_groceries',
 'funeral expenses',
 'gambling',
 'gifts_donations',
 'government/tax/finance',
 'health_care',
 'holidays',
 'housing',
 'industry',
 'insurance',
 'justice/prison',
 'justice_costs',
 'kids',
 'pension',
 'room_rental',
 'service_provider',
 'shopping',
 'sport',
 'transportation',
 'utilities',
 'wellbeing'}

In [17]:
lens=[len(x) for x in data.index]

In [18]:
lens[0]

41

In [21]:
max_len=max(lens)
max_len

48

In [22]:
num_classes=len(set(data))

In [23]:
num_classes

29

# Pythorch model

In [12]:
alphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"

In [186]:
args={'alphabet':alphabet,'number_of_characters':len(alphabet),'max_length':50,'dropout_input':0.2,'number_of_classes':num_classes}

In [187]:
args

{'alphabet': 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-,;.!?:\'"/\\|_@#$%^&*~`+-=<>()[]{}',
 'dropout_input': 0.2,
 'max_length': 50,
 'number_of_characters': 95,
 'number_of_classes': 29}

In [188]:
class CharacterLevelCNN(nn.Module):
    def __init__(self, args):
        super(CharacterLevelCNN, self).__init__()

        # define conv layers

        self.dropout_input = nn.Dropout2d(args['dropout_input'])

        self.conv1 = nn.Sequential(nn.Conv1d(args['number_of_characters'],
                                             256,
                                             kernel_size=7,
                                             padding=0),
                                   nn.ReLU(),
                                   #nn.MaxPool1d(3)
                                   )

        self.conv2 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=7, padding=0),
                                   nn.ReLU(),
                                   #nn.MaxPool1d(3)
                                   )
      
        self.conv3 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=3, padding=0),
                                   nn.ReLU()
                                   )
        
        self.conv4 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=3, padding=0),
                                   nn.ReLU()
                                   )

        self.conv5 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=3, padding=0),
                                   nn.ReLU()
                                   )

        self.conv6 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=3, padding=0),
                                   nn.ReLU(),
                                   nn.MaxPool1d(3)
                                   )
        
        # compute the  output shape after forwarding an input to the conv layers

        input_shape = (128,
                       args['max_length'],
                       args['number_of_characters'])
        
        self.output_dimension = self._get_conv_output(input_shape)

        # define linear layers

        self.fc1 = nn.Sequential(
            nn.Linear(self.output_dimension, 100),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.fc2 = nn.Sequential(
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.fc3 = nn.Linear(100, args['number_of_classes'])

        # initialize weights

        self._create_weights()

    # utility private functions

    def _create_weights(self, mean=0.0, std=0.05):
        for module in self.modules():
            if isinstance(module, nn.Conv1d) or isinstance(module, nn.Linear):
                module.weight.data.normal_(mean, std)


    def _get_conv_output(self, shape):
        x = torch.rand(shape)
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        
        x = x.view(x.size(0), -1)
        output_dimension = x.size(1)
        return output_dimension

    # forward

    def forward(self, x):
        x = self.dropout_input(x)
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [183]:
conv1 = nn.Sequential(nn.Conv1d(args['number_of_characters'],
                                             256,
                                             kernel_size=7,
                                             padding=0),
                                   nn.ReLU(),
                                   #nn.MaxPool1d(3)
                                   )

conv2 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=7, padding=0),
                                   nn.ReLU(),
                                   nn.MaxPool1d(3)
                                   )

conv3=nn.Sequential(nn.Conv1d(256, 256, kernel_size=3, padding=0),
                                   nn.ReLU()
                                   )


In [168]:
def get_conv_output( shape):
        x = torch.rand(shape)
        x = x.transpose(1, 2)
        x = conv1(x)
        x = conv2(x)
        x = conv3(x)
        """
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        """
        #x = x.view(x.size(0), -1)
        output_dimension = x.size()
        return output_dimension

In [165]:
input_shape = (128,
                       50,
                       args['number_of_characters'])

In [172]:
x = torch.rand(input_shape)

In [173]:
x.size()

torch.Size([128, 50, 95])

In [181]:
x.transpose(1,2).size()

torch.Size([128, 95, 50])

In [184]:
y=conv1(x.transpose(1,2))
y.size()

torch.Size([128, 256, 44])

In [185]:
z=conv2(y)
z.size()

torch.Size([128, 256, 12])

In [171]:
get_conv_output(input_shape)

torch.Size([128, 256, 2])

In [190]:
class MyDataset(Dataset):
    def __init__(self, texts, labels, args):
        self.texts = texts
        self.labels = labels
        self.length = len(self.texts)

        self.vocabulary = args['alphabet'] 
        self.number_of_characters = args['number_of_characters'] 
        self.max_length = args['max_length']
        self.identity_mat = np.identity(self.number_of_characters)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        raw_text = self.texts[index]

        data = np.array([self.identity_mat[self.vocabulary.index(i)] for i in list(raw_text)[::-1] if i in self.vocabulary],
                        dtype=np.float32)
        if len(data) > self.max_length:
            data = data[:self.max_length]
        elif 0 < len(data) < self.max_length:
            data = np.concatenate(
                (data, np.zeros((self.max_length - len(data), self.number_of_characters), dtype=np.float32)))
        elif len(data) == 0:
            data = np.zeros(
                (self.max_length, self.number_of_characters), dtype=np.float32)

        label = self.labels[index]
        data = torch.Tensor(data)

        return data, label

### Focal Loss

In [34]:
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): 
            self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): 
            self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, x, target):
        if x.dim()>2:
            x = x.view(x.size(0),x.size(1),-1)  # N,C,H,W => N,C,H*W
            x = x.transpose(1,2)    # N,C,H*W => N,H*W,C
            x = x.contiguous().view(-1,x.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(x)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=x.data.type():
                self.alpha = self.alpha.type_as(x.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: 
            return loss.mean()
        else: 
            return loss.sum()

In [70]:
def process_text(array):
    max_len=args['max_length']
    l=len(array)
    v=[]
    for i in range(l):
      v.append(array[i]+(max_len-len(array[i]))*" ")
    return np.array(v)

In [147]:
data.index=process_text(data.index)

In [148]:
len(data.index[-1])

100

In [36]:
def labels_map(y):
    S=set(y)
    num=len(S)
    dic={word:i for i, word in enumerate(S)}
    
    labels=y.apply(lambda val: dic[val])
    
    return labels.values    

In [37]:
labels=labels_map(data)

In [40]:
labels.shape

(1731940,)

In [41]:
labels

array([21,  4, 21, ...,  8, 28, 21])

In [44]:
data.index

Index(['! Bien Buro Installatie en Netwerken B.V.', '! Manna',
       '! zaak ! MAGEry ', '!Bewust sociaal op web', '!DRAMA! ',
       '!Frank van Bovene', '!M Infra Management', '!R!D!B! Timmerwerken ',
       '!T-Sounds ', '!ThingsthatMatter ',
       ...
       'één0negen ', 'éénKEUKEN', 'être design ', 'être occupé ',
       'ông cà lem ', 'önder Biskin', 'özmes', 'ø audiovisueel',
       'ŠKODA Autohaas', '€5,- Shop '],
      dtype='object', name='Name', length=1731940)

# Train

In [191]:
def train(model, train_loader, epochs, criterion, optimizer, device):
    
    for epoch in range(1, epochs + 1):
        model.train() # Make sure that the model is in training mode.

        total_loss = 0
        iterator=notebook.tqdm(train_loader)
        for batch in iterator:
            # get data
            batch_x, batch_y = batch
            
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()

            # get predictions from model
            y_pred = model(batch_x)
        
            # perform backprop
            loss = criterion(y_pred, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
        iterator.close()
        print("Epoch: {}, Loss: {}".format(epoch, total_loss / len(train_loader)))

In [43]:
from sklearn.model_selection import train_test_split

In [192]:
x_train,x_test,y_train,y_test=train_test_split(data.index,labels,test_size=0.2)

In [193]:
dataset_train=MyDataset(x_train,y_train,args)

In [194]:
dataset_test=MyDataset(x_test,y_test,args)

In [200]:
test_dl=torch.utils.data.DataLoader(dataset_test, batch_size=5000)
train_dl=torch.utils.data.DataLoader(dataset_train, batch_size=5000)

In [199]:
args

{'alphabet': 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-,;.!?:\'"/\\|_@#$%^&*~`+-=<>()[]{}',
 'dropout_input': 0.2,
 'max_length': 50,
 'number_of_characters': 95,
 'number_of_classes': 29}

## Initialize model and train

In [96]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device {}.".format(device))

Using device cuda.


In [201]:
model=CharacterLevelCNN(args).to(device)

In [202]:
optimizer = optim.Adam(model.parameters(),lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [42]:
optimizer = optim.Adam(model.parameters(),lr=0.0001)
loss_fn = FocalLoss(gamma=2)

In [205]:
train(model, train_dl, 3, loss_fn,optimizer, device)


Epoch: 1, Loss: 1.5450930260925841



Epoch: 2, Loss: 1.5338516008082053



Epoch: 3, Loss: 1.5247703743495529


In [112]:
 def accuracy(model,val_dl):
  model.eval()
  with torch.no_grad():
        accu=0
        for batch in val_dl:
            val_X,val_y=batch
            val_X=val_X.to(device)
            val_y=val_y.to(device)
            x=model(val_X)
            probs=F.softmax(x,dim=1)
            preds=probs.argmax(dim=1)
            accu+=(preds==val_y).sum().item()
    
  accu=accu/len(val_dl.dataset)*100
  return accu

In [206]:
accuracy(model,test_dl)

55.41964502234489

In [ ]:
20 epochs: 89.74575900345698
30 epochs: 89.80118733316802

10 epochs: 84.93078751987397

20 epochs: 87.42360371661537